In [2]:
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt   # this lets us visualize the shape files in python...
import folium

os.getcwd()
os.chdir('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis')
# Reading all the files of interest...

metro_rail = gpd.read_file('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis/Metro_Rail_Lines_Stops/Metro_Rail_Lines_Stops.shp')
metro_bus = gpd.read_file('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis/Metro_Bus_Lines/Metro_Bus_Lines.shp')
la_cities = gpd.read_file('C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis/Neighborhood_Councils_(Certified)/Neighborhood_Councils_(Certified).shp')
NC = pd.read_csv('Neighborhood_Councils_(Certified).csv')  

# so I am trying to join NCName from NC dataframe to 311 dataframe so that it
# will be easier for me to plot region wise...

NC_subset = NC[['NAME', 'NC_ID', 'SERVICE_RE']]
NC_subset= NC_subset.rename(columns={'NC_ID':'NC'})

# converting 'NC_ID' in NC from int to float...
NC_subset['NC'] =NC_subset['NC'].astype(float)

# so far these two cols can be joined...so col NC_ID in la_cities and NC in df
# good to go..

# Moving on, going to use just the 311 data and create shape file using the 
# lat and the long...

from shapely.geometry import Point, Polygon 
df_311_2021 = pd.read_csv('MyLA311_Service_Request_Data_2021.csv',low_memory=False)
# 'when you set low_memory=False, it does not give the error anymore. 
#  I guess this is happening because the now of records is close to
#  1.5000000... Also, setting the dtype to 'unicode' did not work...
#  Do not do that...

crs = la_cities.crs
geometry_2021= [Point(xy) for xy in zip(df_311_2021['Longitude'], df_311_2021['Latitude'])]
geo_df_2021 = gpd.GeoDataFrame(df_311_2021, crs= crs, geometry=geometry_2021)

df_merge_2021 = pd.merge(geo_df_2021, NC_subset, on="NC")

reg6 = la_cities[la_cities['SERVICE_RE']=='REGION 6 - CENTRAL 2']
metro_bus_reg6 = gpd.overlay(metro_bus, reg6, how='intersection')
metro_rail_reg6 = gpd.overlay(metro_rail, reg6, how='intersection')

df_2021 = df_merge_2021.loc[df_merge_2021['SERVICE_RE']=='REGION 6 - CENTRAL 2']
df_2021_ssi= df_2021[df_2021['RequestType']=='Single Streetlight Issue']

# heat map
from folium import plugins

m= reg6.explore(
    column= 'NAME', # make choropleth based on 'NC name' column
    name='NC: region 6-Central 2', 
    tooltip='NAME', # show 'NC name' value in tooltip (on hover)
    color="red", # use red color on all points
    popup=True, # show all values in popup (on click)
    tiles="openstreetmap", # use "openstreetmap" tiles
    cmap="Set1", # use "Set1" matplotlib colormap
    style_kwds=dict(color="black") # use black outline
     )
    
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in df_2021_ssi.geometry]
# location should consist of 2 numerical values..
plugins.HeatMap(heat_data, name='Heat map for Single Streetlight Issues').add_to(m) 

data=df_2021_ssi.explore(
     m=m, # pass the map object
     marker_type='circle',
     name='Req Type- Single Streetlight Issue',
     tooltip='RequestType',
     marker_kwds=dict(radius=3, fill=True, opacity=1, weight = 1, color='red')
     )

metro_rail_reg6.explore(
     m=m, # pass the map object
     marker_type='marker',
     marker_kwds={'icon': folium.map.Icon(icon='train', prefix='fa')},
     tooltip="Station", # show "station" column in the tooltip
     tooltip_kwds=dict(labels=True), # do not show column label in the tooltip
     name="Metro Rail stops" # name of the layer in the map
     )
metro_rail_reg6.explore(
     m=m, # pass the map object
     marker_type='circle',
     name='Geofencing',
     marker_kwds=dict(radius=96.012, fill=True, opacity=0.1, weight = 1),
     tooltip="Station", # show "station" column in the tooltip
     tooltip_kwds=dict(labels=True), # do not show column label in the tooltip
     )
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
#folium.TileLayer('Stamen Water Color').add_to(m)
#folium.TileLayer('cartodbpositron').add_to(m)
#folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)

In [3]:
m
